# 1.Setup

In [2]:
!pip install python-binance pandas mplfinance


In [3]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


# Fetching and cleaning data

In [4]:
from binance import Client ,ThreadedDepthCacheManager, ThreadedWebsocketManager
import pandas as pd

In [5]:
import requests
import csv

In [6]:
import requests
import pandas as pd

def fetch_crypto_data():
    """
    Fetches the top 50 cryptocurrency data from CoinGecko and Binance APIs 
    and returns a Pandas DataFrame.
    """
    # Step 1: Fetch top cryptocurrencies from CoinGecko
    coingecko_url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 78,  # Fetch extra to compensate for missing ones
        "page": 1,
        "sparkline": False
    }

    response = requests.get(coingecko_url, params=params)

    # Step 2: Validate API response
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return None

    try:
        top_cryptos = response.json()  # Convert response to JSON
    except ValueError:
        print("Error: Unable to parse JSON. Response:", response.text)
        return None

    if not isinstance(top_cryptos, list):
        print("Unexpected response format:", top_cryptos)
        return None

    # Step 3: Fetch Binance data
    binance_url = "https://api.binance.com/api/v3/ticker/24hr"
    crypto_data = []

    for crypto in top_cryptos:
        if not isinstance(crypto, dict):
            continue

        symbol = crypto.get("symbol", "").upper() + "USDT"

        binance_response = requests.get(binance_url, params={"symbol": symbol})

        if binance_response.status_code == 200:
            try:
                binance_data = binance_response.json()
            except ValueError:
                continue

            if isinstance(binance_data, dict) and "lastPrice" in binance_data:
                crypto_info = {
                    "Cryptocurrency Name": crypto.get("name", "N/A"),
                    "Symbol": symbol,
                    "Current Price (USD)": float(binance_data["lastPrice"]),
                    "Market Cap $": float(crypto.get('market_cap', 0)),
                    "24-hour Trading Volume": float(binance_data.get('quoteVolume', 0)),
                    "24-hour Price Change (%)": float(binance_data.get('priceChangePercent', 0))
                }
                crypto_data.append(crypto_info)

        if len(crypto_data) == 50:  # Stop after collecting 50 cryptos
            break

    # Convert to DataFrame
    df = pd.DataFrame(crypto_data)

    return df


In [7]:
crypto_df = fetch_crypto_data()
crypto_df

,Cryptocurrency Name,Symbol,Current Price (USD),Market Cap $,24-hour Trading Volume,24-hour Price Change (%)
0,Bitcoin,BTCUSDT,85106.910000,1.687898e+12,1.265464e+09,0.452
1,Ethereum,ETHUSDT,2196.340000,2.640031e+11,9.350282e+08,1.504
2,XRP,XRPUSDT,2.224900,1.292182e+11,3.231168e+08,3.556
3,BNB,BNBUSDT,603.250000,8.785265e+10,9.678629e+07,0.490
4,Solana,SOLUSDT,140.760000,7.147415e+10,3.809300e+08,0.306
5,USDC,USDCUSDT,1.000300,5.634432e+10,3.743226e+08,-0.010
6,Dogecoin,DOGEUSDT,0.202410,2.994422e+10,1.219713e+08,0.737
7,Cardano,ADAUSDT,0.647400,2.326118e+10,6.401940e+07,2.550
8,TRON,TRXUSDT,0.234600,2.019501e+10,5.514221e+07,0.773
9,Wrapped Bitcoin,WBTCUSDT,84993.250000,1.098643e+10,1.581210e+07,0.524


In [8]:

crypto_df.index = range(1, len(crypto_df)+1)
crypto_df



,Cryptocurrency Name,Symbol,Current Price (USD),Market Cap $,24-hour Trading Volume,24-hour Price Change (%)
1,Bitcoin,BTCUSDT,85106.910000,1.687898e+12,1.265464e+09,0.452
2,Ethereum,ETHUSDT,2196.340000,2.640031e+11,9.350282e+08,1.504
3,XRP,XRPUSDT,2.224900,1.292182e+11,3.231168e+08,3.556
4,BNB,BNBUSDT,603.250000,8.785265e+10,9.678629e+07,0.490
5,Solana,SOLUSDT,140.760000,7.147415e+10,3.809300e+08,0.306
6,USDC,USDCUSDT,1.000300,5.634432e+10,3.743226e+08,-0.010
7,Dogecoin,DOGEUSDT,0.202410,2.994422e+10,1.219713e+08,0.737
8,Cardano,ADAUSDT,0.647400,2.326118e+10,6.401940e+07,2.550
9,TRON,TRXUSDT,0.234600,2.019501e+10,5.514221e+07,0.773
10,Wrapped Bitcoin,WBTCUSDT,84993.250000,1.098643e+10,1.581210e+07,0.524


In [9]:
row_type= crypto_df.iloc[1].apply(type)
row_type


Cryptocurrency Name                   <class 'str'>
Symbol                                <class 'str'>
Current Price (USD)         <class 'numpy.float64'>
Market Cap $                <class 'numpy.float64'>
24-hour Trading Volume      <class 'numpy.float64'>
24-hour Price Change (%)    <class 'numpy.float64'>
Name: 2, dtype: object

# Data Analysis

#### > Top 5 cryptocurrencies by market cap

In [10]:
crypto_df["Market Cap $"] = crypto_df["Market Cap $"].replace('[\$,]', '', regex=True).astype(float)
top_5 = crypto_df.nlargest(5,'Market Cap $')
top_5

,Cryptocurrency Name,Symbol,Current Price (USD),Market Cap $,24-hour Trading Volume,24-hour Price Change (%)
1,Bitcoin,BTCUSDT,85106.9100,1.687898e+12,1.265464e+09,0.452
2,Ethereum,ETHUSDT,2196.3400,2.640031e+11,9.350282e+08,1.504
3,XRP,XRPUSDT,2.2249,1.292182e+11,3.231168e+08,3.556
4,BNB,BNBUSDT,603.2500,8.785265e+10,9.678629e+07,0.490
5,Solana,SOLUSDT,140.7600,7.147415e+10,3.809300e+08,0.306


#### > Calculating the average current price of Top 50 Cryptocurrencies

In [11]:

Average_price = crypto_df["Current Price (USD)"].mean()
Average_price


np.float64(3517.382807022)

#### Highest and lowest 24-hour percentage price 

In [12]:

highest_change = crypto_df.loc[crypto_df["24-hour Price Change (%)"].idxmax()]
lowest_change = crypto_df.loc[crypto_df["24-hour Price Change (%)"].idxmin()]



In [13]:
highest_change

Cryptocurrency Name                  Hedera
Symbol                             HBARUSDT
Current Price (USD)                 0.24583
Market Cap $                  10347074633.0
24-hour Trading Volume      175994870.94834
24-hour Price Change (%)              6.076
Name: 11, dtype: object

In [14]:
lowest_change

Cryptocurrency Name               Litecoin
Symbol                             LTCUSDT
Current Price (USD)                 118.53
Market Cap $                  8959006297.0
24-hour Trading Volume      91708312.41952
24-hour Price Change (%)            -5.335
Name: 15, dtype: object

In [15]:
from openpyxl import load_workbook
import time

In [16]:


def update_excel(df, file_name="crypto_data.xlsx"):
    try:
        # Load existing workbook
        book = load_workbook(file_name)
        writer = pd.ExcelWriter(file_name, engine="openpyxl", mode="a", if_sheet_exists="replace")
    except FileNotFoundError:
        # Create a new file if not found
        writer = pd.ExcelWriter(file_name, engine="openpyxl")

    # Write data
    df.to_excel(writer, sheet_name="Live Data", index=False)

    # Save file
    writer.close()
    print("Excel updated successfully.")



In [ ]:
# Simulate fetching and updating every 5 minutes
while True:
    # Fetch latest data (use your existing fetching function)
    df = fetch_crypto_data()  # Replace with your function

    # Update the Excel file
    if df is not None:
        update_excel(df)
        
    # Wait for 5 minutes before updating again
    time.sleep(300)


    


Excel updated successfully.
